In [1]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('h4', 's-title').text
    source = card.find("span", 's-source').text
    posted = card.find('span', 's-time').text.replace('·', '').strip()
    description = card.find('p', 's-desc').text.strip()
    raw_link = card.find('a').get('href')
    unquoted_link = requests.utils.unquote(raw_link)
    pattern = re.compile(r'RU=(.+)\/RK')
    clean_link = re.search(pattern, unquoted_link).group(1)
    
    article = (headline, source, posted, description, clean_link)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://news.search.yahoo.com/search?p={}'
    url = template.format(search)
    articles = []
    links = set()
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'NewsArticle')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            link = article[-1]
            if not link in links:
                links.add(link)
                articles.append(article)        
                
        # find the next page
        try:
            url = soup.find('a', 'next').get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('result.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Source', 'Posted', 'Description', 'Link'])
        writer.writerows(articles)
        
    return articles

In [2]:
articles = get_the_news('INTEL CORPORATION')

In [3]:
articles

[('Intel (INTC) Directed to Pay $2.18B Fine After Losing Patent Case',
  'Zacks via Yahoo Finance',
  '4 hours ago',
  'Intel Corporation INTC recently was ordered by a federal court in Texas to shell out $2.18 billion...',
  'https://finance.yahoo.com/news/intel-intc-directed-pay-2-134601250.html'),
 ('The Zacks Analyst Blog Highlights: Tesla, Intel, Verizon, Costco and Intuit',
  'Zacks via Yahoo Finance',
  '1 day ago',
  'Every day the Zacks Equity Research analysts discuss the latest news and events impacting stocks and...',
  'https://finance.yahoo.com/news/zacks-analyst-blog-highlights-tesla-122312529.html'),
 ("Intel Corporation's (NASDAQ:INTC) Stock Is Going Strong: Is the Market Following Fundamentals?",
  'Simply Wall St. via Yahoo Finance',
  '5 days ago',
  'Intel (NASDAQ:INTC) has had a great run on the share market with its stock up by a significant 27%...',
  'https://finance.yahoo.com/news/intel-corporations-nasdaq-intc-stock-064323654.html'),
 ('These are America’s 10